# INSTALL DEPENDENCIES

In [5]:
!pip install groq python-dotenv

In [6]:
import os
from groq import Groq
from dotenv import load_dotenv

# API Key Setup

In [7]:
from getpass import getpass

# Prompt user for API key if not already set
if "GROQ_API_KEY" not in os.environ:
    os.environ["GROQ_API_KEY"] = getpass("Enter your GROQ API Key: ")

# Initialize Groq client
client = Groq(api_key=os.environ["GROQ_API_KEY"])

print("Groq client initialized successfully.")

Enter your GROQ API Key: ··········
Groq client initialized successfully.


# Define Expert Configurations

In [20]:
MODEL_NAME = "llama-3.3-70b-versatile"

MODEL_CONFIG = {
    "technical": {
        "system_prompt": """You are a Technical Support Expert.
Be precise, rigorous, and code-focused.
Provide clear debugging steps and example fixes.
Avoid unnecessary conversation.
"""
    },
    "billing": {
        "system_prompt": """You are a Billing Support Specialist.
Be empathetic, professional, and policy-driven.
Focus on charges, refunds, subscriptions, and invoices.
Provide clear next steps for resolution.
"""
    },
    "general": {
        "system_prompt": """You are a helpful customer support assistant.
Handle general questions and casual conversation politely.
"""
    }
}

# Router Logic

In [21]:
def route_prompt(user_input):
    router_prompt = f"""
Classify the following user request into one of these categories:
[technical, billing, general]

Return ONLY the category name. No explanation.

User request:
{user_input}
"""

    response = client.chat.completions.create(
        model=MODEL_NAME,
        messages=[
            {"role": "system", "content": "You are a strict intent classifier."},
            {"role": "user", "content": router_prompt}
        ],
        temperature=0
    )

    category = response.choices[0].message.content.strip().lower()
    return category

# Orchestrator Logic

In [22]:
def process_request(user_input):

    # Step 1: Route
    category = route_prompt(user_input)
    print(f"Routed to: {category}")

    # Fallback safety
    if category not in MODEL_CONFIG:
        category = "general"

    system_prompt = MODEL_CONFIG[category]["system_prompt"]

    # Step 2: Call Expert
    response = client.chat.completions.create(
        model=MODEL_NAME,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_input}
        ],
        temperature=0.7
    )

    return response.choices[0].message.content

# User End Point

In [23]:
user_query = input("Enter your customer support request: ")
response = process_request(user_query)

print("\nFinal Response:\n")
print(response)

Enter your customer support request: I was charged twice for my subscription this month.
Routed to: billing

Final Response:

I'm so sorry to hear that you were charged twice for your subscription this month. I can imagine how frustrating that must be for you. I'm here to help resolve this issue as quickly as possible.

To get started, could you please provide me with your subscription details, including your account name and the date of the duplicate charge? This will help me locate your account and investigate the issue further.

Additionally, I'll need to verify some information to ensure that I'm assisting the correct account holder. Please confirm your email address associated with the subscription and the last 4 digits of the payment method used for the subscription.

Once I have this information, I'll be happy to review your account and work with you to process a refund for the duplicate charge. If the duplicate charge was an error on our part, we'll also take steps to prevent t